In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_community.chat_models import ChatOllama
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver 

In [3]:
load_dotenv()

True

In [4]:
llm = ChatOllama(model="llama2:7b", temperature=0)

C:\Users\sdais\AppData\Local\Temp\ipykernel_3824\1205151563.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama2:7b", temperature=0)


In [5]:
class JokeState(TypedDict):

    topic:str
    joke:str
    explanation: str

In [6]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [7]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [8]:

graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [9]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1) # type:ignore

{'topic': 'pizza',
 'joke': 'Why did the pizza go to therapy? It was feeling a little crusty!',
 'explanation': '\nAh, I see! The joke is a play on words, with "crusty" having a double meaning. In one sense, it can refer to the crispy exterior of a pizza crust. But in another sense, "crusty" can also mean grumpy or irritable, as in "he\'s feeling a little crusty today." So, the pizza is going to therapy because it\'s feeling a bit grumpy or irritable, which is a clever and unexpected twist on the typical reason for seeing a therapist (e.g., dealing with personal issues). I hope that helps clarify things!'}

In [10]:
workflow.get_state(config1) # type:ignore

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? It was feeling a little crusty!', 'explanation': '\nAh, I see! The joke is a play on words, with "crusty" having a double meaning. In one sense, it can refer to the crispy exterior of a pizza crust. But in another sense, "crusty" can also mean grumpy or irritable, as in "he\'s feeling a little crusty today." So, the pizza is going to therapy because it\'s feeling a bit grumpy or irritable, which is a clever and unexpected twist on the typical reason for seeing a therapist (e.g., dealing with personal issues). I hope that helps clarify things!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092162-8e7c-6dbe-8002-21f94b16e23d'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-15T09:27:12.170131+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092161-fc92-64ca-8001-b93593d64f96'}}, ta

In [11]:
list(workflow.get_state_history(config1)) # type:ignore

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? It was feeling a little crusty!', 'explanation': '\nAh, I see! The joke is a play on words, with "crusty" having a double meaning. In one sense, it can refer to the crispy exterior of a pizza crust. But in another sense, "crusty" can also mean grumpy or irritable, as in "he\'s feeling a little crusty today." So, the pizza is going to therapy because it\'s feeling a bit grumpy or irritable, which is a clever and unexpected twist on the typical reason for seeing a therapist (e.g., dealing with personal issues). I hope that helps clarify things!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092162-8e7c-6dbe-8002-21f94b16e23d'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-15T09:27:12.170131+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092161-fc92-64ca-8001-b93593d64f96'}}, t

In [12]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2) # type:ignore

{'topic': 'pasta',
 'joke': 'Why did the spaghetti refuse to get dressed? Because it already had a saucy attitude!',
 'explanation': '\nThe joke "Why did the spaghetti refuse to get dressed? Because it already had a saucy attitude!" is a play on words that combines two different meanings of the word "saucy."\n\nIn one sense, "saucy" can refer to something that is impudent or impertinent, as in "The spaghetti was being saucy and refused to get dressed." In this context, the joke is funny because it takes a common phrase ("refused to get dressed") and gives it an unexpected twist by replacing the word "refused" with "saucy," which has a different meaning.\n\nHowever, there is also a second layer of humor in the joke. In culinary terms, "saucy" can refer to food that is covered in sauce or gravy. So, the punchline "because it already had a saucy attitude" could be interpreted as the spaghetti being too coated in sauce to bother getting dressed. This double meaning adds an extra layer of h

## Fault Tolerance

In [13]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [14]:
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [15]:
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]} # type:ignore 

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"} # type:ignore

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True} # type:ignore

In [16]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}}) # type:ignore
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [2]:
# # 6. Re-run to show fault-tolerant resume
# print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
# final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
# print("\n✅ Final State:", final_state)